In [1]:
import torch
from torch import nn
from peft import LoraConfig,get_peft_model,PeftModel

# 1. 自定义模型适配

In [2]:
net1 = nn.Sequential(
    nn.Linear(10,10),
    nn.ReLU(),
    nn.Linear(10,2)
)
net1

Sequential(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=2, bias=True)
)

In [4]:
for name,parm in net1.named_parameters():
    print(name)
    

0.weight
0.bias
2.weight
2.bias


In [5]:
config = LoraConfig(target_modules=["0"])


In [6]:
model1 = get_peft_model(net1,config)

In [7]:
model1

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=10, bias=True)
        (lora_dropout): ModuleDict(
          (default): Identity()
        )
        (lora_A): ModuleDict(
          (default): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (default): Linear(in_features=8, out_features=10, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
        (lora_magnitude_vector): ModuleDict()
      )
      (1): ReLU()
      (2): Linear(in_features=10, out_features=2, bias=True)
    )
  )
)

# 2. 多适配器加载与切换

In [9]:
net2 = nn.Sequential(
    nn.Linear(10,10),
    nn.ReLU(),
    nn.Linear(10,2)
)
net2

Sequential(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): ReLU()
  (2): Linear(in_features=10, out_features=2, bias=True)
)

In [10]:
config1 = LoraConfig(target_modules=["0"])
model2 = get_peft_model(net2,config1)
model2.save_pretrained("./loraA")

In [11]:
config2 = LoraConfig(target_modules=["2"])
model2 = get_peft_model(net2, config2)
model2.save_pretrained("./loraB")

/Users/liuzhen/Documents/PEFT_Study/.venv/lib/python3.12/site-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/Users/liuzhen/Documents/PEFT_Study/.venv/lib/python3.12/site-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [12]:
model2 = PeftModel.from_pretrained(net2, model_id="./loraA/", adapter_name="loraA")
model2

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=10, bias=True)
        (lora_dropout): ModuleDict(
          (default): Identity()
          (loraA): Identity()
        )
        (lora_A): ModuleDict(
          (default): Linear(in_features=10, out_features=8, bias=False)
          (loraA): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (default): Linear(in_features=8, out_features=10, bias=False)
          (loraA): Linear(in_features=8, out_features=10, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
        (lora_magnitude_vector): ModuleDict()
      )
      (1): ReLU()
      (2): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=2, bias=True)
        (lora_dropout): ModuleDict(
          (default): Identity()
        )
        (lora_A): ModuleDict(
    

In [13]:
model2.load_adapter("./loraB/", adapter_name="loraB")
model2

PeftModel(
  (base_model): LoraModel(
    (model): Sequential(
      (0): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=10, bias=True)
        (lora_dropout): ModuleDict(
          (default): Identity()
          (loraA): Identity()
        )
        (lora_A): ModuleDict(
          (default): Linear(in_features=10, out_features=8, bias=False)
          (loraA): Linear(in_features=10, out_features=8, bias=False)
        )
        (lora_B): ModuleDict(
          (default): Linear(in_features=8, out_features=10, bias=False)
          (loraA): Linear(in_features=8, out_features=10, bias=False)
        )
        (lora_embedding_A): ParameterDict()
        (lora_embedding_B): ParameterDict()
        (lora_magnitude_vector): ModuleDict()
      )
      (1): ReLU()
      (2): lora.Linear(
        (base_layer): Linear(in_features=10, out_features=2, bias=True)
        (lora_dropout): ModuleDict(
          (default): Identity()
          (loraB): Identity()
        )
    

In [14]:
model2.active_adapter

'loraA'

In [15]:
model2(torch.arange(0, 10).view(1, 10).float())

tensor([[-0.8964,  2.6760]])

In [16]:
for name, param in model2.named_parameters():
    print(name, param)

base_model.model.0.base_layer.weight Parameter containing:
tensor([[-0.1441,  0.2309,  0.2151, -0.3134, -0.0033,  0.2736, -0.1951,  0.1742,
          0.1468,  0.1074],
        [-0.2579, -0.0858,  0.1584, -0.0578, -0.2392, -0.0630,  0.2318, -0.2393,
         -0.0062,  0.0786],
        [ 0.1186, -0.2464, -0.2950, -0.0779, -0.0255, -0.1746,  0.2648, -0.1534,
          0.1297,  0.1081],
        [ 0.1264,  0.2960, -0.1411,  0.2337,  0.3121,  0.2827, -0.0189,  0.0353,
          0.0972, -0.0306],
        [ 0.0834,  0.1453,  0.0923,  0.0661,  0.1951,  0.1751, -0.2713, -0.2658,
          0.2839,  0.1610],
        [ 0.1076, -0.0019,  0.2880, -0.0242,  0.1195, -0.1445,  0.1648,  0.2779,
          0.1720,  0.2818],
        [ 0.1466,  0.1509,  0.0417,  0.1879, -0.2056,  0.2546,  0.0467,  0.0109,
          0.0031,  0.2971],
        [-0.1008, -0.0935, -0.0917, -0.2641,  0.2559,  0.1507,  0.0146,  0.2908,
          0.2180,  0.2246],
        [ 0.0124, -0.2925, -0.1521,  0.1501,  0.2793,  0.0559,  0.166

In [17]:
for name, param in model2.named_parameters():
    if name in ["base_model.model.0.lora_A.loraA.weight", "base_model.model.0.lora_B.loraA.weight"]:
        param.data = torch.ones_like(param)

In [18]:
model2(torch.arange(0, 10).view(1, 10).float())

tensor([[ 77.5160, 149.8932]])

In [19]:
model2.set_adapter("loraB")

In [20]:
model2.active_adapter

'loraB'

In [23]:
model2(torch.arange(0,10).view(1,10).float())

tensor([[-0.8964,  2.6760]], grad_fn=<AddBackward0>)

# 禁用适配器

In [25]:
model2.set_adapter("loraA")

In [26]:
model2(torch.arange(0,10).view(1,10).float())

tensor([[ 77.5160, 149.8932]], grad_fn=<AddmmBackward0>)

In [28]:
with model2.disable_adapter():
    print(model2(torch.arange(0,10).view(1,10).float()))

tensor([[-0.8964,  2.6760]])
